# Tidy Data

What makes data "tidy"?
1. Each variable must have its own column
2. Each observation must have its own row
3. Each value must have its own cell

---

## Columns Contain Values, Not Variables 

This section addresses one of the common issues when dealing with messy data: Our columns headers aren't actually variables but instead values.

### Keep On Column Fixed

To illustrate the concept of each variable having its own column, we'll use some data from Pew. 

In this dataset, not every column is a variable. There is a column for the religion variable, but the data for income is stratified and spread across different columns. This is good for presenting data but for analytics, we'll want to reshape. 

In [19]:
import pandas as pd
pew = pd.read_csv("./pew.csv")

In [20]:
pew

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116
5,Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529
6,Hindu,1,9,7,9,11,34,47,48,54,37
7,Historically Black Prot,228,244,236,238,197,223,131,81,78,339
8,Jehovah's Witness,20,27,24,24,21,30,15,11,6,37
9,Jewish,19,19,25,25,30,95,69,87,151,162


The reason this data isn't considered tidy is because the column names for every column except religion, aren't really variables. These column names are just categories but they're categories of what could be a single variable, income. So, to fix this, we can create a new variable called income where the current column names are potential values of that new variable.

Presumably, the unit of observation for this dataset is the number of households for each religion and income category. So, we'll want to ultimately have an observation for each combination of religion and income. This ensures that each observation in the dataset only has one measured value, the count of household per combination.

Thus, we'll end up with two independent variables, religion and income, and one dependent variable. This jives with how we might normalize data to reduce the functional dependencies in the data. 

This current view is called "wide data". We'll want to turn it into "long tidy data". To do so we'll unpivot/melt/gather our dataframe. 

For the purposes of this notebook, "pivot longer" = the `.melt()` dataframe method and "pivot wider" = the `.pivot()` method. 

The `.melt()` method is a Pandas DataFrame method that is used to reshape a dataframe into a tidy format:
* `id_vars` - container representing the variables that will remain as is
* `value_vars` - the columns to be melted down (turned into long tidy data)
* `var_name` - string for the new column name when `value_vars` *isn't specified*
* `value_name` - string for new column name that represents the values for `var_name`

In [21]:
# melt down all the columns except for religion to make the data long tidy
pew_long = pew.melt(id_vars='religion')
pew_long

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


This small dataset is now "tidy"; each variable has its own column and each unit of observation has its own row. Observer, the value for each of these observations is the original value in the dataset.

However, to improve on this even more, we can name the newly created variable and the value of each row. So, we'll name the newly melted variable income and the original value for each of those observations count.

In [22]:
# recreate tidied dataset but with names
pew_long = pew.melt(id_vars="religion", var_name="income", value_name="count")
pew_long

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


### Keep Multiple Columns Fixed

In the previous example, we kept one column fixed while creating a new column for the spread out data. 

In this next example, we'll see how to keep multiple columns from the original dataset in place.

In [23]:
billboard = pd.read_csv("./billboard.csv")
billboard

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,86,83.0,77.0,74.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,85,83.0,83.0,82.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,95,94.0,91.0,85.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,99,99.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Just looking at it initially, this dataset isn't tidy because the weeks could be their own variable where the possible values of a week variable could be the current column headers. We'd then use the value of the combination of year, artist, track, time, date.entered, and week to have the value be the measurement for which spot on the billboard each unit. 

There is also an issue that the numerical values for the original dataset aren't in the same format. E.g. In observation 0, the wk1 value is an integer while the remaining values contain a decimal. 

The following cell keeps our desired variables in tact but melts the remaining into one named variable.

In [24]:
billboard_long = billboard.melt(
    id_vars=["year", "artist", "track", "time", "date.entered"],
    var_name="week", 
    value_name="rating",
)

billboard_long

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0
...,...,...,...,...,...,...,...
24087,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,wk76,NaN
24088,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,wk76,NaN
24089,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,wk76,NaN
24090,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,wk76,NaN


So, to hold multiple columns, we simply use a list for the `id_vars` parameter.

---

## Columns Contain Multiple Variables

This section deals with another common issue when dealing with messy data: the columns may contain more than one variable. 

In [25]:
ebola = pd.read_csv("./country_timeseries.csv")
ebola.columns

Index(['Date', 'Day', 'Cases_Guinea', 'Cases_Liberia', 'Cases_SierraLeone',
       'Cases_Nigeria', 'Cases_Senegal', 'Cases_UnitedStates', 'Cases_Spain',
       'Cases_Mali', 'Deaths_Guinea', 'Deaths_Liberia', 'Deaths_SierraLeone',
       'Deaths_Nigeria', 'Deaths_Senegal', 'Deaths_UnitedStates',
       'Deaths_Spain', 'Deaths_Mali'],
      dtype='object')

The issue here is that the `Cases_X` and `Deaths_X` contain what should be two variables:
1. The number of cases or number of deaths 
2. The country name (`X` is a stand in)

To tidy this dataset, we can try to fix the first two columns (`day` and `date`) and then we can try to melt the rest such that each unit of observation gets its own row.

In [26]:
# melt the other columns while holding the remaining fixed (no named variable)
ebola_long = ebola.melt(
    id_vars=["Date", "Day"]
)

ebola_long

,Date,Day,variable,value
0,1/5/2015,289,Cases_Guinea,2776.0
1,1/4/2015,288,Cases_Guinea,2775.0
2,1/3/2015,287,Cases_Guinea,2769.0
3,1/2/2015,286,Cases_Guinea,NaN
4,12/31/2014,284,Cases_Guinea,2730.0
...,...,...,...,...
1947,3/27/2014,5,Deaths_Mali,NaN
1948,3/26/2014,4,Deaths_Mali,NaN
1949,3/25/2014,3,Deaths_Mali,NaN
1950,3/24/2014,2,Deaths_Mali,NaN


### Split and Add Columns Individually

While the above implementation works somewhat, we can still break down the columns further. We can break down the melted column into cases/deaths and the country name. It's not always the case, but we can do so for this example by splitting on the underscore.

We'll have to use some string parsing using the `.str` accessor and make a call to the Python string object's `.split()` method. 

In [27]:
# access the string methods to split on a delimiter
variable_split = ebola_long.variable.str.split('_')
variable_split

0       [Cases, Guinea]
1       [Cases, Guinea]
2       [Cases, Guinea]
3       [Cases, Guinea]
4       [Cases, Guinea]
             ...       
1947     [Deaths, Mali]
1948     [Deaths, Mali]
1949     [Deaths, Mali]
1950     [Deaths, Mali]
1951     [Deaths, Mali]
Name: variable, Length: 1952, dtype: object

After breaking up the strings into status and country pairs, we can use `.str.get()` to retrieve the portion of the list we want.

In [28]:
# extract the first element of each list for status 
status_values = variable_split.str.get(0)

# extract the second element of each list for country names 
country_values = variable_split.str.get(1)

In [29]:
status_values

0        Cases
1        Cases
2        Cases
3        Cases
4        Cases
         ...  
1947    Deaths
1948    Deaths
1949    Deaths
1950    Deaths
1951    Deaths
Name: variable, Length: 1952, dtype: object

In [30]:
country_values

0       Guinea
1       Guinea
2       Guinea
3       Guinea
4       Guinea
         ...  
1947      Mali
1948      Mali
1949      Mali
1950      Mali
1951      Mali
Name: variable, Length: 1952, dtype: object

Basically all we did was just create vectors of the data we wanted and now we're going to want to make them their own variables in our dataframe.

In [31]:
ebola_long['status'] = status_values
ebola_long['country'] = country_values
ebola_long

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea
...,...,...,...,...,...,...
1947,3/27/2014,5,Deaths_Mali,NaN,Deaths,Mali
1948,3/26/2014,4,Deaths_Mali,NaN,Deaths,Mali
1949,3/25/2014,3,Deaths_Mali,NaN,Deaths,Mali
1950,3/24/2014,2,Deaths_Mali,NaN,Deaths,Mali


### Split and Combine in a Single Step

We can do the above steps in a single step. The `.split()` method has the `expand` parameter that will return a DataFrame where each result of the split is automatically in a separate column. 

This is a MUCH more convenient way of doing what we did above.

In [32]:
# reset our ebola_long data
ebola_long = ebola.melt(id_vars=["Date", "Day"])

# split melted column by _ 
variable_split = ebola_long.variable.str.split('_', expand=True)

variable_split

,0,1
0,Cases,Guinea
1,Cases,Guinea
2,Cases,Guinea
3,Cases,Guinea
4,Cases,Guinea
...,...,...
1947,Deaths,Mali
1948,Deaths,Mali
1949,Deaths,Mali
1950,Deaths,Mali


In [33]:
# multiple assign to create the new columns in the dataset 
ebola_long[['status', 'country']] = variable_split
ebola_long

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea
...,...,...,...,...,...,...
1947,3/27/2014,5,Deaths_Mali,NaN,Deaths,Mali
1948,3/26/2014,4,Deaths_Mali,NaN,Deaths,Mali
1949,3/25/2014,3,Deaths_Mali,NaN,Deaths,Mali
1950,3/24/2014,2,Deaths_Mali,NaN,Deaths,Mali


--- 

## Variables in Both Rows and Columns 

There may be cases where one of our columns actually holds two variables instead of one variable. When this occurs, we'll have to "pivot" the variable into separate columns, so basically doing the opposite of what we've done with the other two cases. We're going from long data to wide data. 

In [34]:
weather = pd.read_csv("./weather.csv")
weather

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,36.3,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,33.2,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18.2,NaN,NaN,NaN,NaN


This issue here is that again, the day "variables" aren't actually variables and we should melt them down into one guy where these column headers are the values. The other issue is that the element column contains variables that should probably be expanded into their own variables. 

For example, each day should only have one minimum temperate and one maximum temperature. This is makes sense but temperature data for each day currently uses two rows for each.

In [35]:
weather_melt = weather.melt(
    id_vars=["id", "year", "month", "element"], 
    var_name="day", 
    value_name="temp",
)

weather_melt

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
...,...,...,...,...,...,...
677,MX17004,2010,10,tmin,d31,NaN
678,MX17004,2010,11,tmax,d31,NaN
679,MX17004,2010,11,tmin,d31,NaN
680,MX17004,2010,12,tmax,d31,NaN


Next, we'll need to expand that column out. 

In [36]:
weather_tidy = weather_melt.pivot_table(
    index=['id', 'year', 'month', 'day'],  # index w/these in order
    columns='element',  # keys to the grouped pivot table
    values='temp'  # column to aggregate
)

weather_tidy

element                 tmax  tmin
id      year month day            
MX17004 2010 1     d30  27.8  14.5
             2     d11  29.7  13.4
                   d2   27.3  14.4
                   d23  29.9  10.7
                   d3   24.1  14.4
             3     d10  34.5  16.8
                   d16  31.1  17.6
                   d5   32.1  14.2
             4     d27  36.3  16.7
             5     d27  33.2  18.2
             6     d17  28.0  17.5
                   d29  30.1  18.0
             7     d3   28.6  17.5
                   d14  29.9  16.5
             8     d23  26.4  15.0
                   d5   29.6  15.8
                   d29  28.0  15.3
                   d13  29.8  16.5
                   d25  29.7  15.6
                   d31  25.4  15.4
                   d8   29.0  17.3
             10    d5   27.0  14.0
                   d14  29.5  13.0
                   d15  28.7  10.5
                   d28  31.2  15.0
                   d7   28.1  12.9
             11    d2   31.3  16.3
                   d5   26.3   7.9
                   d27  27.7  14.2
                   d26  28.1  12.1
                   d4   27.2  12.0
             12    d1   29.9  13.8
                   d6   27.8  10.5

Not going to lie, this one was a bit more confusing than the others to understand the solution. 